In [2]:
from rdkit import Chem
from rdkit.Chem import rdDistGeom
from rdkit.Chem import AllChem
import numpy as np
import copy
import sys
import os
import importlib

sys.path.append(snakemake.params.rdkit_path)
# TODO: double check this is all sensible..
# TODO: should not rely on OMEGA structure...

In [3]:
full_module = f"{snakemake.wildcards.rdkit_type}"
generator = importlib.import_module(full_module)

In [4]:
# generator.conf_generator()

In [5]:
# def etkdgv3(smile, numConf, params=None, getParams=None):
#     from rdkit import Chem
#     from rdkit.Chem import AllChem

#     if getParams is None:
#         getParams = False

#     # Set ETKDG parameters
#     if params is None:
#         #params = rdDistGeom.srETKDGv3()
#         params = rdDistGeom.ETKDGv3()
#         params.useRandomCoords = True
#         params.numThreads = 0
#         params.maxAttempts = 2000
#         params.pruneRmsThresh = 0.5

#     if getParams:
#         parameters = dir(params)
#         parameters = [x for x in parameters if '__' not in x]
#         parameters = [x for x in parameters if 'Set' not in x]
#         parameter_val = [getattr(params, x) for x in parameters]
#         param_dict = dict(zip(parameters, parameter_val))
#         return param_dict

#     # Create molecule
#     mol = Chem.MolFromSmiles(smile)
#     Chem.SanitizeMol(mol)

#     # Add H's
#     mol = Chem.AddHs(mol, addCoords=True, explicitOnly=True) #, addResidueInfo=True)


#     # Produce conformers
#     AllChem.EmbedMultipleConfs(mol, numConfs=numConf, params=params)

#     return mol,params

In [6]:
with open(snakemake.input.omega_smile) as f:
    smile = f.read()
smile = smile[:-1]
smile

In [7]:
# why is omega the reference molecule here?
ref_mol = Chem.MolFromPDBFile(
    snakemake.input.omega,
    removeHs=False,
)

In [10]:
?Chem.MolFromPDBFile

In [8]:
ref_mol.RemoveAllConformers()

In [9]:
ref_mol

In [10]:
ref_mol.GetNumAtoms()

In [11]:
smile = Chem.MolToSmiles(ref_mol)
smile

In [12]:
# Generate conformers...
confs, params = generator.conf_generator(smile, 400)

In [13]:
parameters = dir(params)
parameters = [x for x in parameters if "__" not in x]
parameters = [x for x in parameters if "Set" not in x]
parameter_val = [getattr(params, x) for x in parameters]
param_dict = dict(zip(parameters, parameter_val))
param_dict

In [14]:
confs.GetNumConformers()

In [15]:
confs

In [16]:
confs.GetNumAtoms()

In [17]:
# Chem.MolToPDBFile(confs, snakemake.output.pdb)

In [18]:
AllChem.AssignBondOrdersFromTemplate(ref_mol, confs)

In [19]:
hit_ats = list(confs.GetSubstructMatch(ref_mol))
# Find missing atom indices.
missing = set(range(confs.GetNumAtoms())) - set(hit_ats)
if missing == set():
    order = list(confs.GetSubstructMatches(ref_mol)[0])
    mol = Chem.RenumberAtoms(confs, order)
else:
    # Now remove these atoms
    mw = Chem.RWMol(confs)
    for i in missing:
        mw.RemoveAtom(i)
    order = list(mw.GetSubstructMatch(ref_mol))
    Chem.RenumberAtoms(mw, order)
    mol = mw

In [20]:
missing

In [21]:
new_mol = copy.deepcopy(ref_mol)

In [22]:
for i in range(mol.GetNumConformers()):
    new_mol.AddConformer(mol.GetConformer(i))

In [23]:
new_mol.GetNumConformers()

In [24]:
Chem.MolToPDBFile(new_mol, snakemake.output.pdb)

In [32]:
results = Chem.rdForceFieldHelpers.MMFFOptimizeMoleculeConfs(
    new_mol, maxIters=2000, numThreads=snakemake.threads
)
energies = [en for conv, en in results]
energies = np.array(energies)
np.savetxt(snakemake.output.mmff_energies, energies)